In [1]:
!pip install music21

In [2]:
pip install --upgrade music21==6.7.1

Requirement already up-to-date: music21==6.7.1 in /home/workspace/.local/lib/python3.8/site-packages (6.7.1)
Note: you may need to restart the kernel to use updated packages.


In [6]:
from music21 import converter,instrument,note,chord,stream
import glob
import pickle
import numpy as np
import tensorflow as tf
from keras.utils import np_utils

## Read a Midi File

In [7]:
midi = converter.parse("midi_songs/EyesOnMePiano.mid")

In [8]:
midi

<music21.stream.Score 0x7fbb69e0a160>

In [9]:
midi.show('midi')

In [11]:
elements_to_parse = midi.flat.notes

In [12]:
len(elements_to_parse)

1425

In [14]:
elements_to_parse[6]

<music21.note.Note F#>

In [15]:
elements_to_parse[6].pitch

<music21.pitch.Pitch F#6>

In [16]:
str(elements_to_parse[3].pitch)

'A6'

In [17]:
elements_to_parse[68]

<music21.chord.Chord F#4 C#5>

In [18]:
elements_to_parse[68].normalOrder

[1, 6]

In [19]:
"+".join(str(n) for n in elements_to_parse[68].normalOrder)

'1+6'

In [20]:
isinstance(elements_to_parse[3],note.Note)

True

In [21]:
isinstance(elements_to_parse[68],note.Note)

False

In [22]:
isinstance(elements_to_parse[68],chord.Chord)

True

In [23]:
notes_demo = []


for ele in elements_to_parse:
    # If the element is a Note,then store its pitch.
    if isinstance(ele,note.Note):
        notes_demo.append(str(ele.pitch))
    
    
    # If the element is a Chord, split each note of chord and join them with '+'
    elif isinstance(ele,chord.Chord):
        notes_demo.append("+".join(str(n) for n in ele.normalOrder))

## Preprocessing all Files

In [25]:
notes = []
for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file) # Converts file into stream.Score Object
    
    #print("parsing %s"%file)
    
    elements_to_parse = midi.flat.notes
    
    for ele in elements_to_parse:
        # If the element is a Note,then store its pitch.
        if isinstance(ele,note.Note):
            notes.append(str(ele.pitch))


        # If the element is a Chord, split each note of chord and join them with '+'
        elif isinstance(ele,chord.Chord):
            notes.append("+".join(str(n) for n in ele.normalOrder))
    

In [26]:
len(notes)

60866

In [27]:
with open("notes","wb") as filepath:
    pickle.dump(notes,filepath)

In [28]:
notes = None

In [29]:
print(notes)

None


In [30]:
with open("notes","rb") as filepath:
    notes = pickle.load(filepath)

In [31]:
print(len(notes))

60866


In [32]:
len(set(notes))

359

In [33]:
n_vocab = len(set(notes))

In [34]:
print("Total notes- ",len(notes))
print("Unique notes- ",n_vocab)

Total notes-  60866
Unique notes-  359


### Prepare Sequential Data For LSTM

In [35]:
# How many elements LSTM input should consider.
sequence_length = 100

In [36]:
pitchnames = sorted(set(notes))

In [37]:
# Mapping between elements to integer values.
ele_to_int = dict((ele,num) for num,ele in enumerate(pitchnames))

In [38]:
network_input = []
network_output = []

In [39]:
for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length] # Contains 100 elements.
    seq_out = notes[i+sequence_length]
    
    # seq_in and seq_out contain strings. But LSTM accepts only numerical data.
    # So, we convert the strings to numbers using the mapping 'ele_to_int' which we created above.
    
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out]) # seq_out is a single element.

In [40]:
# No of examples
n_patterns = len(network_input)
print(n_patterns)

60766


In [41]:
# For feeding these data in LSTM, we need to convert them into numpy.
network_input = np.reshape(network_input,(n_patterns,sequence_length,1)) # 1 is used because LSTM receives data in 3 dimensions.
print(network_input.shape)

(60766, 100, 1)


In [42]:
#network_input

In [43]:
# Normalise the data
normalised_network_input = network_input/float(n_vocab)

In [44]:
#normalised_network_input

In [47]:
# One Hot Encoding of network_ouput
network_output = np_utils.to_categorical(network_output)
print(network_output.shape)
print(network_output)

(60766, 359)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Create Model

In [48]:
from keras.models import Sequential,load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [49]:
model = Sequential()
model.add(LSTM(units=512,
              input_shape=(normalised_network_input.shape[1],normalised_network_input.shape[2]),  # shape-> (100,1)
              return_sequences=True)) 
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation='softmax'))

model.summary()

2022-03-19 14:10:36.798138: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-19 14:10:36.798203: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-19 14:10:36.798239: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (anurag1905): /proc/driver/nvidia/version does not exist
2022-03-19 14:10:36.815863: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1052672   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 512)          2099200   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_2 (LSTM)               (None, 512)               2099200   
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0

In [50]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "new_weights.hdf5",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="min",
)

model_hist = model.fit(normalised_network_input,network_output,epochs=100,batch_size=64,callbacks=[checkpoint])

In [67]:
model = load_model("new_weights.hdf5")

## Predictions

In [68]:
sequence_length = 100
network_input = []
for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length] # Contains 100 elements.
    # seq_in contain strings. But LSTM accepts only numerical data.
    # So, we convert the strings to numbers using the mapping 'ele_to_int'.
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [87]:
start = np.random.randint(len(network_input)-1) # Choose a random data point from network_input.    

In [88]:
start

33794

In [89]:
# Mapping of integer to element.
int_to_ele = dict((num,ele) for num,ele in enumerate(pitchnames))

# Initial pattern
pattern = network_input[start]
prediction_output = []

# Generate 200 elements
for note_index in range(200):
    prediction_input = np.reshape(pattern,(1,len(pattern),1))
    prediction_input = prediction_input/float(n_vocab)
    
    prediction = model.predict(prediction_input,verbose=0)
    
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_output.append(result)
    
    pattern.append(idx)
    pattern = pattern[1:]

In [90]:
print(len(prediction_output))

200


## Create Midi File

In [92]:
prediction_output[4] # This is a Note

'G2'

In [93]:
offset = 0
new_note = note.Note(prediction_output[4]) # Creates a note object
new_note.offset =  offset
new_note.storedInstrument = instrument.Piano()

In [94]:
offset = 0 # Time
output_notes = []

for pattern in prediction_output:
    
    # if the pattern is a chord.
    
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note)) # Create Note object for each note in the chord.
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
        
        new_chord = chord.Chord(temp_notes) # creates the chord object from the list of notes.
        new_chord.offset = offset
        output_notes.append(new_chord)
        
    
    # if the pattern is a note.
    else:
        new_note = note.Note(pattern) # Creates a note object
        new_note.offset =  offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset+=0.5

In [96]:
# create a stream object from the generated notes
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi',fp='generated_music.mid')

'generated_music.mid'

In [97]:
midi_stream.show('midi')